In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob # For finding result files
from IPython.display import display, HTML

# --- Configuration ---
# Define the path to your results directory (where the dated subfolder is)
# This should match 'results_dir_timedated' from your experiment script
BASE_RESULTS_DIR = r"G:\My Drive\code_projects\btc_forecast\results" # <<< UPDATE THIS TO YOUR LOCAL PATH
FREQUENCY = "1h" # <<< Ensure this matches the frequency of results you want to analyze
DATE_TODAY_RESULTS = "20250507" # <<< UPDATE THIS to the date folder of your results

RESULTS_PATH = os.path.join(BASE_RESULTS_DIR, f"{FREQUENCY}_{DATE_TODAY_RESULTS}")

# Number of initial OOS chunks to average for "Initial OOS Score"
NUM_INITIAL_OOS_CHUNKS_FOR_AVG = 3

print(f"Results Path: {RESULTS_PATH}")
pd.set_option('display.float_format', '{:.6f}'.format)

Results Path: G:\My Drive\code_projects\btc_forecast\results\1h_20250507


In [2]:
json_pattern = os.path.join(RESULTS_PATH, f"*_{FREQUENCY}_*_healthspan_summary.json")
result_files = glob.glob(json_pattern)

all_results_data = []
if not result_files:
    print(f"No result files found matching pattern: {json_pattern}")
else:
    print(f"Found {len(result_files)} result files to load.")
    for file_path in result_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Each file contains a list of results (one per candle_count for a specific asset/target)
                all_results_data.extend(data)
        except Exception as e:
            print(f"Error loading or parsing {file_path}: {e}")

if not all_results_data:
    print("No data loaded. Please check RESULTS_PATH and file contents.")
    # Stop execution or handle error
    raise ValueError("No results data loaded.")
else:
    all_results_df = pd.DataFrame(all_results_data)
    print(f"\nSuccessfully loaded {len(all_results_df)} records into a DataFrame.")
    display(all_results_df.head())
    print("\nDataFrame Info:")
    all_results_df.info()

Found 2536 result files to load.

Successfully loaded 24820 records into a DataFrame.


,target,asset,candle_count,best_score_val_internal,error,time_taken_seconds,num_data_points_train,num_val_windows_used,external_eval_wql_scores,healthspan_steps,wql_threshold
0,volume,1000APU_USDT_USDT,100,-0.034787,None,5.834485,100,10,"[{'time_since_train': 360000.0, 'WQL': -0.0308...",200,-0.020907
1,volume,1000APU_USDT_USDT,250,-0.034902,None,3.453808,250,10,"[{'time_since_train': 360000.0, 'WQL': -0.0308...",200,-0.021022
2,volume,1000APU_USDT_USDT,500,-0.035334,None,5.225691,500,10,"[{'time_since_train': 360000.0, 'WQL': -0.0308...",200,-0.021454
3,volume,1000APU_USDT_USDT,750,-0.035781,None,6.834671,750,10,"[{'time_since_train': 360000.0, 'WQL': -0.0308...",200,-0.021901
4,volume,1000APU_USDT_USDT,1000,-0.036428,None,9.028275,1000,10,"[{'time_since_train': 360000.0, 'WQL': -0.0308...",200,-0.022548



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24820 entries, 0 to 24819
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   target                    24820 non-null  object 
 1   asset                     24820 non-null  object 
 2   candle_count              24820 non-null  int64  
 3   best_score_val_internal   24820 non-null  float64
 4   error                     0 non-null      object 
 5   time_taken_seconds        24820 non-null  float64
 6   num_data_points_train     24820 non-null  int64  
 7   num_val_windows_used      24820 non-null  int64  
 8   external_eval_wql_scores  24820 non-null  object 
 9   healthspan_steps          24820 non-null  int64  
 10  wql_threshold             24810 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 2.1+ MB


In [3]:
def process_results(df):
    processed_records = []
    for _, row in df.iterrows():
        record = row.to_dict()

        # 1. Convert internal score (-WQL) to actual IS WQL (lower is better)
        if pd.notna(record.get("best_score_val_internal")):
            record['actual_is_wql'] = -record['best_score_val_internal']
        else:
            record['actual_is_wql'] = np.nan

        # 2. Process external_eval_wql_scores
        oos_wql_scores_finite = [] # Store only finite scores
        if isinstance(record.get("external_eval_wql_scores"), list):
            for score_entry in record["external_eval_wql_scores"]:
                if isinstance(score_entry, dict):
                    wql_val = score_entry.get("WQL")
                    # Ensure the value is a number and is finite (not NaN, inf, -inf)
                    if isinstance(wql_val, (int, float, np.number)) and np.isfinite(wql_val):
                        oos_wql_scores_finite.append(float(wql_val))

        if oos_wql_scores_finite: # Check if the list of finite scores is not empty
            # 3. Initial OOS WQL Mean (from first N finite chunks)
            record['initial_oos_wql_mean'] = np.mean(oos_wql_scores_finite[:NUM_INITIAL_OOS_CHUNKS_FOR_AVG]) # np.mean is fine now

            # 4. OOS WQL Standard Deviation (stability)
            if len(oos_wql_scores_finite) >= 2: # Std dev needs at least 2 points
                record['oos_wql_std'] = np.std(oos_wql_scores_finite) # np.std is fine now
            elif len(oos_wql_scores_finite) == 1:
                record['oos_wql_std'] = 0.0 # Std dev of a single point is 0
            else: # Should not happen if oos_wql_scores_finite is not empty, but as safeguard
                record['oos_wql_std'] = np.nan
        else:
            record['initial_oos_wql_mean'] = np.nan
            record['oos_wql_std'] = np.nan

        # 5. IS-OOS Gap
        if pd.notna(record['actual_is_wql']) and pd.notna(record['initial_oos_wql_mean']):
            record['is_oos_gap'] = record['initial_oos_wql_mean'] - record['actual_is_wql']
        else:
            record['is_oos_gap'] = np.nan

        processed_records.append(record)

    return pd.DataFrame(processed_records)

if 'all_results_df' in locals() and not all_results_df.empty:
    processed_df = process_results(all_results_df.copy()) # Apply the updated function
    print("Processed DataFrame with new metrics:")
    display_cols_processed = [
        'asset', 'target', 'candle_count',
        'actual_is_wql', 'initial_oos_wql_mean', 'is_oos_gap', 'oos_wql_std',
        'healthspan_steps', 'wql_threshold', 'error'
    ]
    # Ensure columns exist before trying to display them
    display_cols_present = [col for col in display_cols_processed if col in processed_df.columns]
    if display_cols_present:
        display(processed_df[display_cols_present].head())
    else:
        display(processed_df.head()) # Fallback to display all if key columns are missing
else:
    print("Skipping processing as 'all_results_df' is not available.")

Processed DataFrame with new metrics:


,asset,target,candle_count,actual_is_wql,initial_oos_wql_mean,is_oos_gap,oos_wql_std,healthspan_steps,wql_threshold,error
0,1000APU_USDT_USDT,volume,100,0.034787,-0.024602,-0.059390,0.009253,200,-0.020907,None
1,1000APU_USDT_USDT,volume,250,0.034902,-0.024602,-0.059504,0.009253,200,-0.021022,None
2,1000APU_USDT_USDT,volume,500,0.035334,-0.024602,-0.059937,0.009253,200,-0.021454,None
3,1000APU_USDT_USDT,volume,750,0.035781,-0.024602,-0.060383,0.009253,200,-0.021901,None
4,1000APU_USDT_USDT,volume,1000,0.036428,-0.024602,-0.061030,0.009253,200,-0.022548,None


In [4]:
# @title Analysis 3: Overall Best Effort Candle Count & Associated Healthspan

if 'processed_df' in locals() and not processed_df.empty:
    print("\n--- Analysis 3: Overall Best Effort Candle Count by Combined Criteria ---")

    overall_summary_list = []
    # Group by asset and target, as requested
    grouped_for_best_effort = processed_df.groupby(['asset', 'target'])

    for (asset, target), group_df_orig in grouped_for_best_effort:
        if group_df_orig.empty:
            print(f"Skipping {asset} - {target}: Empty group.")
            continue

        group_df = group_df_orig.copy() # Work on a copy to avoid SettingWithCopyWarning

        # Filter for rows where all necessary metrics for ranking are available
        metrics_for_ranking = ['initial_oos_wql_mean', 'is_oos_gap', 'oos_wql_std']
        # Also need these for reporting
        reporting_metrics = ['candle_count', 'healthspan_steps', 'actual_is_wql', 'wql_threshold']
        
        # Keep only rows where all ranking metrics are non-NaN
        group_df.dropna(subset=metrics_for_ranking, inplace=True)

        if group_df.empty:
            print(f"[{asset}/{target}] Skipping: Not enough valid data after NaN drop for ranking metrics.")
            overall_summary_list.append({
                'asset': asset, 'target': target,
                'Best_Effort_CC': np.nan, 'Initial_OOS_WQL': np.nan,
                'IS_OOS_Gap': np.nan, 'OOS_WQL_Std': np.nan,
                'Healthspan_at_Best_Effort_CC': np.nan,
                'Actual_IS_WQL_at_Best_Effort_CC': np.nan,
                'WQL_Threshold_at_Best_Effort_CC': np.nan,
                'Comment': 'Insufficient data for ranking'
            })
            continue

        # Calculate ranks for each criterion (lower value of metric = better rank)
        # 'min' method assigns the same rank to tied values
        group_df['rank_initial_oos'] = group_df['initial_oos_wql_mean'].rank(method='min', na_option='bottom')
        group_df['abs_is_oos_gap'] = group_df['is_oos_gap'].abs() # We rank based on absolute gap
        group_df['rank_is_oos_gap'] = group_df['abs_is_oos_gap'].rank(method='min', na_option='bottom')
        group_df['rank_oos_std'] = group_df['oos_wql_std'].rank(method='min', na_option='bottom')

        # Composite score (sum of ranks - lower composite rank is better)
        group_df['composite_rank'] = group_df['rank_initial_oos'] + \
                                     group_df['rank_is_oos_gap'] + \
                                     group_df['rank_oos_std']

        # Find the row with the best composite rank
        # In case of ties in composite_rank, further sort by:
        # 1. initial_oos_wql_mean (lower is better)
        # 2. oos_wql_std (lower is better)
        # 3. is_oos_gap (absolute, lower is better)
        # 4. candle_count (prefer smaller candle_count in case of further ties, optional)
        best_overall_row = group_df.sort_values(
            by=['composite_rank', 'initial_oos_wql_mean', 'oos_wql_std', 'abs_is_oos_gap', 'candle_count']
        ).iloc[0]

        overall_summary_list.append({
            'asset': asset,
            'target': target,
            'Best_Effort_CC': int(best_overall_row['candle_count']),
            'Initial_OOS_WQL': best_overall_row['initial_oos_wql_mean'],
            'IS_OOS_Gap': best_overall_row['is_oos_gap'], # Report original gap
            'OOS_WQL_Std': best_overall_row['oos_wql_std'],
            'Healthspan_at_Best_Effort_CC': best_overall_row['healthspan_steps'],
            'Actual_IS_WQL_at_Best_Effort_CC': best_overall_row['actual_is_wql'],
            'WQL_Threshold_at_Best_Effort_CC': best_overall_row['wql_threshold'],
            'Composite_Rank': best_overall_row['composite_rank'], # For reference/debugging
            'Comment': '' # For any notes
        })

    if overall_summary_list:
        final_overall_summary_df = pd.DataFrame(overall_summary_list)
        print("\n--- Overall 'Best Effort' Candle Count Summary Table ---")
        print("This table shows one 'best effort' candle_count per asset/target, chosen by ranking performance across multiple criteria.")
        
        # Select and order columns for the final display
        display_summary_cols = [
            'asset', 'target', 'Best_Effort_CC',
            'Actual_IS_WQL_at_Best_Effort_CC', # Added for context
            'Initial_OOS_WQL', 'IS_OOS_Gap', 'OOS_WQL_Std',
            'Healthspan_at_Best_Effort_CC',
            'WQL_Threshold_at_Best_Effort_CC', # Added for context
            'Comment'
        ]
        # Ensure all selected columns exist to prevent KeyError
        display_summary_cols = [col for col in display_summary_cols if col in final_overall_summary_df.columns]

        display(final_overall_summary_df[display_summary_cols])  # (HTML(final_overall_summary_df[display_summary_cols].to_html(na_rep="-")))
    else:
        print("Could not generate the overall best effort summary table. No valid data after grouping or ranking.")
else:
    print("Skipping Analysis 3 (Overall Best Effort) as 'processed_df' is not available.")


--- Analysis 3: Overall Best Effort Candle Count by Combined Criteria ---

--- Overall 'Best Effort' Candle Count Summary Table ---
This table shows one 'best effort' candle_count per asset/target, chosen by ranking performance across multiple criteria.


,asset,target,Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Comment
0,1000000BABYDOGE_USDT_USDT,close,100,0.002684,-0.017187,-0.019871,0.013039,1000,0.016875,
1,1000000BABYDOGE_USDT_USDT,high,100,0.002151,-0.003645,-0.005796,0.001136,1000,-0.000446,
2,1000000BABYDOGE_USDT_USDT,low,100,0.001755,-0.015294,-0.017048,0.010813,1000,0.014464,
3,1000000BABYDOGE_USDT_USDT,open,100,0.002575,-0.004416,-0.006992,0.001900,1000,0.000274,
4,1000000BABYDOGE_USDT_USDT,volume,100,0.010788,-0.013204,-0.023992,0.008426,1000,0.001851,
...,...,...,...,...,...,...,...,...,...,...
2530,ZRX_USDT_USDT,close,1000,0.003347,-0.017134,-0.020480,0.012788,1000,0.015834,
2531,ZRX_USDT_USDT,high,1000,0.002721,-0.007030,-0.009751,0.003752,1000,0.002907,
2532,ZRX_USDT_USDT,low,1000,0.002711,-0.014697,-0.017408,0.009898,1000,0.012136,
2533,ZRX_USDT_USDT,open,1000,0.003370,-0.006940,-0.010310,0.002520,1000,0.000409,


In [5]:
final_overall_summary_df.describe()

,Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Composite_Rank
count,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2535.000000,2534.000000,2535.000000
mean,345.700197,-0.013927,-0.021028,0.009162,885.562130,0.007101,0.006652,3.340039
std,313.119556,0.011884,0.019470,0.008988,295.300988,0.009910,0.012173,1.390985
min,100.000000,-0.224060,-0.355353,0.000000,0.000000,0.000003,-0.051112,3.000000
25%,100.000000,-0.018813,-0.024725,0.002921,1000.000000,0.002124,0.000093,3.000000
50%,250.000000,-0.012081,-0.016237,0.007553,1000.000000,0.003136,0.004010,3.000000
75%,500.000000,-0.005896,-0.008985,0.012615,1000.000000,0.007344,0.013678,3.000000
max,1000.000000,-0.000018,-0.000021,0.108716,1000.000000,0.183626,0.122301,13.000000


In [6]:
final_overall_summary_df["Healthspan_at_Best_Effort_CC"].value_counts()

Healthspan_at_Best_Effort_CC
1000    2173
0        158
100       49
200       38
500       32
300       27
400       22
600       13
800       10
900        8
700        5
Name: count, dtype: int64

In [7]:
vc = final_overall_summary_df["Best_Effort_CC"].value_counts()
display(vc)

Best_Effort_CC
100     1121
250      602
1000     319
500      300
750      193
Name: count, dtype: int64

In [8]:
asset_selection_df = final_overall_summary_df[final_overall_summary_df["Best_Effort_CC"] == vc.index[0]]
asset_selection_df = asset_selection_df[asset_selection_df["Healthspan_at_Best_Effort_CC"] > 0]
hs_vc = asset_selection_df["Healthspan_at_Best_Effort_CC"].value_counts()
display(hs_vc)

Healthspan_at_Best_Effort_CC
1000    1029
100       12
400       10
300        8
500        8
200        7
700        3
600        1
800        1
Name: count, dtype: int64

In [9]:
filtered_asset_sel_df = asset_selection_df[asset_selection_df["Healthspan_at_Best_Effort_CC"] == hs_vc.index[0]]
display(filtered_asset_sel_df.head())
selected_assets = filtered_asset_sel_df["asset"].drop_duplicates().tolist()

,asset,target,Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Composite_Rank,Comment
0,1000000BABYDOGE_USDT_USDT,close,100,-0.017187,-0.019871,0.013039,1000,0.002684,0.016875,3.000000,
1,1000000BABYDOGE_USDT_USDT,high,100,-0.003645,-0.005796,0.001136,1000,0.002151,-0.000446,3.000000,
2,1000000BABYDOGE_USDT_USDT,low,100,-0.015294,-0.017048,0.010813,1000,0.001755,0.014464,3.000000,
3,1000000BABYDOGE_USDT_USDT,open,100,-0.004416,-0.006992,0.001900,1000,0.002575,0.000274,3.000000,
4,1000000BABYDOGE_USDT_USDT,volume,100,-0.013204,-0.023992,0.008426,1000,0.010788,0.001851,3.000000,


In [10]:
# @title Display All Processed Metrics for Specific Candle Counts

# <<< --- DEFINE YOUR CANDLE COUNTS OF INTEREST HERE --- >>>
# For example, if your experiment ran with [100, 250, 500, 750, 1000, 1500, 2000]
# and now you only want to see detailed results for [500, 1000]
specific_candle_counts_to_view = [100, 250, 500] # <<< MODIFY THIS LIST WITH YOUR CHOICES

print(f"Filtering results to show only for candle_counts: {specific_candle_counts_to_view}")

if 'processed_df' in locals() and not processed_df.empty:
    # Filter the DataFrame for the specified candle_counts
    filtered_results_df = processed_df[processed_df['candle_count'].isin(specific_candle_counts_to_view)][display_cols_processed].copy()

    if filtered_results_df.empty:
        print(f"No results found in the loaded data for the specified candle counts: {specific_candle_counts_to_view}")
    else:
        print(f"\nDisplaying all processed columns for selected candle_counts: {specific_candle_counts_to_view}")
        
        # Ensure all columns are displayed by pandas
        # pd.set_option('display.max_columns', None)
        # pd.set_option('display.max_rows', 200) # Show more rows if needed
        # pd.set_option('display.width', 1000)   # Adjust terminal width if needed

        # Sort for easier comparison (optional, but recommended)
        # This will group by asset, then target, then by the selected candle_count
        filtered_results_df_sorted = filtered_results_df.sort_values(
            by=['asset', 'target', 'candle_count']
        ).reset_index(drop=True)
        
        # Display the filtered DataFrame with all its columns
        # Using HTML for better rendering in Jupyter
        display(filtered_results_df_sorted)  # (HTML(filtered_results_df_sorted.to_html(na_rep="-")))
        
        print(f"\nTable above shows all {len(filtered_results_df_sorted.columns)} processed columns for your selected candle counts.")
        print("Relevant columns for your questions include:")
        print("- 'asset', 'target', 'candle_count'")
        print("- 'actual_is_wql' (In-Sample WQL, lower is better)")
        print("- 'initial_oos_wql_mean' (Avg WQL of first few Out-of-Sample chunks, lower is better)")
        print("- 'is_oos_gap' (Difference: Initial OOS WQL - IS WQL, smaller absolute value is better)")
        print("- 'oos_wql_std' (Stability/consistency of OOS WQL, lower is better)")
        print("- 'healthspan_steps' (How long model stayed good, higher is better)")
        print("- 'wql_threshold' (The actual WQL error limit for healthspan)")
        print("- 'best_score_val_internal' (Original -WQL from AutoGluon internal validation, higher is better)")
        print("- 'error' (Any errors during processing)")

else:
    print("Cannot filter results as 'processed_df' is not available. Please ensure previous cells, especially data loading and processing, have run successfully.")

Filtering results to show only for candle_counts: [100, 250, 500]

Displaying all processed columns for selected candle_counts: [100, 250, 500]


,asset,target,candle_count,actual_is_wql,initial_oos_wql_mean,is_oos_gap,oos_wql_std,healthspan_steps,wql_threshold,error
0,1000000BABYDOGE_USDT_USDT,close,100,0.002684,-0.017187,-0.019871,0.013039,1000,0.016875,None
1,1000000BABYDOGE_USDT_USDT,close,100,0.002684,-0.017187,-0.019871,0.013039,1000,0.016875,None
2,1000000BABYDOGE_USDT_USDT,close,250,0.002685,-0.017187,-0.019871,0.013039,1000,0.016874,None
3,1000000BABYDOGE_USDT_USDT,close,250,0.002685,-0.017187,-0.019871,0.013039,1000,0.016874,None
4,1000000BABYDOGE_USDT_USDT,close,500,0.002812,-0.017187,-0.019998,0.013039,1000,0.016747,None
...,...,...,...,...,...,...,...,...,...,...
15045,ZRX_USDT_USDT,volume,100,0.011892,-0.026507,-0.038399,0.012639,1000,0.007067,None
15046,ZRX_USDT_USDT,volume,250,0.013179,-0.026507,-0.039686,0.012639,1000,0.005780,None
15047,ZRX_USDT_USDT,volume,250,0.013179,-0.026507,-0.039686,0.012639,1000,0.005780,None
15048,ZRX_USDT_USDT,volume,500,0.012657,-0.026507,-0.039164,0.012639,1000,0.006302,None



Table above shows all 10 processed columns for your selected candle counts.
Relevant columns for your questions include:
- 'asset', 'target', 'candle_count'
- 'actual_is_wql' (In-Sample WQL, lower is better)
- 'initial_oos_wql_mean' (Avg WQL of first few Out-of-Sample chunks, lower is better)
- 'is_oos_gap' (Difference: Initial OOS WQL - IS WQL, smaller absolute value is better)
- 'oos_wql_std' (Stability/consistency of OOS WQL, lower is better)
- 'healthspan_steps' (How long model stayed good, higher is better)
- 'wql_threshold' (The actual WQL error limit for healthspan)
- 'best_score_val_internal' (Original -WQL from AutoGluon internal validation, higher is better)
- 'error' (Any errors during processing)
